Установим все необходимык библиотеки для парсинга

In [4]:
import pandas as pd
!pip install pubchempy
import pubchempy as pbp
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Считываем изначальный датасет

In [45]:
data = pd.read_excel('7_whit_cid.xlsx')

In [46]:
#Считываем столбец по которому будем искать дескрипторы
smiles = data['CanonicalSMILES']
#Временно удалим дубликаты для того чтобы ускорить парсинг
smiles = smiles.drop_duplicates()
smiles = smiles.reset_index(drop = True)

In [12]:
#Создаем список состоящий из названий всех дескрипторов доступных в pubchem
porp = ['CanonicalSMILES','MolecularFormula', 'MolecularWeight', 'IsomericSMILES', 'InChI', 'InChIKey', 'IUPACName', 'XLogP', 'ExactMass', 'MonoisotopicMass', 'TPSA', 'Complexity', 'Charge', 'HBondDonorCount', 'HBondAcceptorCount', 'RotatableBondCount', 'HeavyAtomCount', 'IsotopeAtomCount', 'AtomStereoCount', 'DefinedAtomStereoCount', 'UndefinedAtomStereoCount', 'BondStereoCount', 'DefinedBondStereoCount', 'UndefinedBondStereoCount', 'CovalentUnitCount', 'Volume3D', 'XStericQuadrupole3D', 'YStericQuadrupole3D', 'ZStericQuadrupole3D', 'FeatureCount3D', 'FeatureAcceptorCount3D', 'FeatureDonorCount3D', 'FeatureAnionCount3D', 'FeatureCationCount3D', 'FeatureRingCount3D', 'FeatureHydrophobeCount3D', 'ConformerModelRMSD3D', 'EffectiveRotorCount3D', 'ConformerCount3D']

In [13]:
#Создадим два пцстых датафрейма. В первый будем записывать полученные дескрипторы а второй будет заменять строку если pubchem
#не сможет найти дескрипторы по конкретному smiles
df = pd.DataFrame(columns=porp)
a = pd.DataFrame(columns=porp)

In [14]:
#Создаем цикл который перебирает все значения в name series
for i in range(len(smiles)):
  try:
    #Парсим все дескрипторы из списка для каждой молекулы
    b = pbp.get_properties(properties = porp, identifier = smiles[i], namespace='smiles', searchtype=None, as_dataframe = True)
    #Записываем полученные значения в датафрейм
    b = pd.DataFrame(b, columns=porp)
    #Добавляем строку с данными в созданный выше датафрейм
    df = pd.concat([df,b], ignore_index=True)
  except:
    #В случае возникновения ошибки добавляем пустую строку
    df = pd.concat([df,a], ignore_index=True)

Проверяем получнный датасет

In [ ]:
df

Считываем колонку изначальную колонку CanonicalSMILES 

In [88]:
smiles_ = data['CanonicalSMILES']
smiles_ = pd.DataFrame(smiles_)

При помощи функции merge востанавливаваем убранные ранее дубликаты. Для это используем left.join а индексной колонкой делаем общую колонку 'CanonicalSMILES'

In [89]:
diskr_pubchem = pd.merge(smiles_, df, how = 'left')


Проверим правильно ли выполнена фунция merge

In [ ]:
diskr_pubchem

Выгружаем полученный датасет, проверяем его и добавляем дескрипторы в изначальный датасет

In [18]:
diskr_pubchem.to_excel('Pubchem_diskr.xlsx', index = False)